# 处理计算

在以 Azure 机器学习作业形式运行脚本时，需要定义作业运行的执行上下文。 一个关键配置是将在其上运行脚本的计算目标。 这可以是本地工作站（在本例中为计算实例），也可以是远程计算目标，例如按需预配的 Azure 机器学习托管计算群集。

在此笔记本中，你将创建计算群集并浏览作业的计算目标。

## 准备工作

需要最新版本的 azureml-ai-ml 包才能运行此笔记本中的代码。 运行下面的单元以验证是否已安装它。

> **注意**：
> 如果未安装“azure-ai-ml”包，请运行 `pip install azure-ai-ml` 以进行安装。

In [ ]:
## 连接到工作区

安装必需的 SDK 包后，就可以连接到工作区了。

若要连接到工作区，我们需要标识符参数 - 订阅 ID、资源组名称和工作区名称。 已为你填写资源组名称和工作区名称。 只需订阅 ID 即可完成命令。

若要查找所需的参数，请单击工作室右上角的订阅和工作区名称。 右侧将打开一个窗格。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 复制订阅 ID，并将“YOUR-SUBSCRIPTION-ID”替换为复制的值。 </p>

## 创建计算群集

许多情况下，本地计算资源可能不足，无法进行需处理大量数据的复杂试验或长期运行的试验；并且你可能想利用在云中动态创建和使用计算资源这一功能。 Azure 机器学习支持一系列计算目标，你可以在工作区中定义这些目标并将其用于运行作业；只需在使用资源时为其付费。

可以在 [Azure 机器学习工作室](https://ml.azure.com)中使用 Python SDK 或 Azure CLI 创建计算群集。 以下代码单元检查工作区中是否存在名称为 `aml-cluster` 的计算群集，如果不存在，则创建它。

In [ ]:
创建计算群集后，只能更改以下项的配置：

- `min_instances`：节点数下限
- `max_instances`：节点数上限
- `idle_time_before_scale_down`：纵向缩减之前的空闲时间

目前你的计算群集 `aml-cluster` 的缩放上限只有一个节点。 我们来将该上限更改为两个节点，以允许并行计算。

In [ ]:
更新计算群集时，可以通过打印其属性来验证其配置。

## 创建脚本来训练模型

若要训练模型，请先在 src 文件夹中创建 diabetes_training.py 脚本 。 该脚本使用训练数据所在的文件夹中的 diabetes.csv 文件。

In [ ]:
## 在计算群集上运行作业

现在即可在创建的计算群集上运行作业。

> **注意**：
> 作业需要一些时间才能启动，因为计算群集需要从零节点扩展至一个节点。 计算群集准备就绪后，将运行脚本。 作业完成后，计算群集将缩减至零个节点。 可以在“计算”页中查看计算群集的状态。

After you've created a compute cluster, you can only change the configuration for:

- `min_instances`: Minimum number of nodes
- `max_instances`: Maximum number of nodes
- `idle_time_before_scale_down`: Idle time before scale down

Currently, your compute cluster `aml-cluster` can only scale do a maximum of one node. Let's change that to two, to allow for parallel compute.

In [ ]:
from azure.ai.ml.entities import AmlCompute

cluster_scale = AmlCompute(
    name="cpu-cluster",
    max_instances=2,
)
ml_client.begin_create_or_update(cluster_scale)

When the compute cluster is updated, you can verify its configuration by printing its attributes.

In [ ]:
cpu_cluster = ml_client.compute.get("cpu-cluster")

print (
        f"AMLCompute with name {cpu_cluster.name} has a maximum of {cpu_cluster.max_instances} nodes"
    )

## Create a script to train a model

To train a model, you'll first create the **diabetes_training.py** script in the **src** folder. The script uses the **diabetes.csv** file in the same folder as the training data.

In [ ]:
%%writefile src/diabetes-training.py
# import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('diabetes.csv')

# separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# set regularization hyperparameter
reg = 0.01

# train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

## Run a job on a compute cluster

Now, you're ready to run the job on the compute cluster you created.

> **Note**:
> The job will take some time to start as the compute cluster will need to scale from zero to one node. Once the compute cluster is ready, the script will be run. When the job has finished, the compute cluster will scale back down to zero nodes. You can review the compute cluster's status in the **Compute** page.

In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="cpu-cluster",
    display_name="diabetes-train-cluster",
    experiment_name="diabetes-training"
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)